# Pipeline and models
The final steps. Prepare the features to be consumed by different models that we'll train and test, and finally present the results

#### Steps:
- [x] Load the basic features from step 03
- [x] Split the dataset into train and test 
- [x] Normalize the data to be trained
- [x] Train and test different models and architectures
- [x] Find the best results and summarize them

#### The pipeline
Polinomial features > Normalization > Train (multiple models)

#### Prerequisites for this notebook:
- Go through `03_feature_engineering.ipynb` so the raw parsed datasets are available 

In [1]:
# First we must mount google drive 
from google.colab import drive

GDRIVE_BASE_PATH = '/content/gdrive'
drive.mount(GDRIVE_BASE_PATH)

# Loading our project setup
HOME_DIR = '/content/gdrive/My Drive/project_scs3253'
% cd $HOME_DIR

from util.project_setup import ProjectSetup
import numpy as np
import pandas as pd

RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)

SCORING_METRICS = 'f1'
PICK_TOP_N_MODELS = 5
CV_COUNT = 3 
POLYNOMIAL_DEGREE = 2

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/project_scs3253


In [2]:
# Loading the dataframe with basic features

#df = pd.read_csv(f'{ProjectSetup.data_dir}/basic_features_dataframe_file.csv', sep=',', index_col=0)

# We are using the files that is already generated by the previos step. It is renamed here to remind us what
# filtering we did. (1000ms holdtime cutoff, 1000 min observation, mild impact only)
df = pd.read_csv(f'{ProjectSetup.data_dir}/filter_1000holdtime_1000obs_mildonly.csv', sep=',', index_col=0)
df = df.drop("userkey", axis=1)

# Show the breakdown to see if we get a balanced dataset
print("Parkinsons:\n", df['parkinsons'].value_counts())

y = df.parkinsons
X = df.drop('parkinsons', axis=1)

Parkinsons:
 True     43
False    41
Name: parkinsons, dtype: int64


In [3]:
from sklearn.model_selection import train_test_split

# Spliting with 20% on the test set - as we don't have lots of data, if we keep more for test, we can't train
# good models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)

print("Parkinsons:\n", y_train.value_counts())

Parkinsons:
 True     34
False    33
Name: parkinsons, dtype: int64


In [0]:
# Creating a wrapper class responsible to delegate the fit/predict 
# calls to the underlying estimator. This decorator helps us to 
# train different ML models on a single pipeline and select the best
from sklearn.base import BaseEstimator
class EstimatorDecorator(BaseEstimator):
  def __init__(self, estimator=None):
    self.estimator = estimator

  def fit(self, X, y=None, **kwargs):
    self.estimator.fit(X, y)
    return self

  def predict(self, X, y=None):
    return self.estimator.predict(X)

  def predict_proba(self, X):
    return self.estimator.predict_proba(X)

  def score(self, X, y):
    return self.estimator.score(X, y)

In [0]:
# Now let's build our pipeline using our decorator classifier
#from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

training_pipeline = Pipeline(verbose=False, steps=[
  ('pol', PolynomialFeatures(degree = POLYNOMIAL_DEGREE)),
  ('nor', Normalizer()),
  ('clf', EstimatorDecorator())
])


In [0]:
# Let's create some smart functions to train all models and report the results
# (we might want to move this later to our `util` module)
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import pprint as pp
import collections
import re

MyEmptydf = pd.DataFrame()

def class_to_string(klass):
  return re.match(r'\A(?P<class_name>\w+)\(', str(klass)).group('class_name')


def calculate_scores(predictions, y):
  
  confusion = confusion_matrix(y, predictions)    
  tn, fp, fn, tp = confusion.ravel()
  specificity = tn / (tn + fp)
  
  scores = {
    'f1'         : f1_score(y, predictions),
    'precision'  : precision_score(y, predictions),
    'recall'     : recall_score(y, predictions),
    'specificity': round(specificity, 6),
    'accuracy'   : accuracy_score(y, predictions),
    'auc'        : roc_auc_score(y, predictions),
    'confusion'  : confusion,
  }
  return scores


def print_scores(scores_tr, scores_te):
  if (scores_te['accuracy']==0) and (scores_te['f1']==0):
    print(f"> Scores     <train>    | <NoTest>")
  else:
    print(f"> Scores     <train>    | <test>")
  print(f"> F1         : {scores_tr['f1']:f} | {scores_te['f1']:f}")
  print(f"> Precision  : {scores_tr['precision']:f} | {scores_te['precision']:f}")
  print(f"> Recall     : {scores_tr['recall']:f} | {scores_te['recall']:f}")
  print(f"> Specificity: {scores_tr['specificity']:f} | {scores_te['specificity']:f}")
  print(f"> Accuracy   : {scores_tr['accuracy']:f} | {scores_te['accuracy']:f}")
  print(f"> AUC        : {scores_tr['auc']:f} | {scores_te['auc']:f}")
  print(f"> ** Confusion matrix (Train)**")
  print(scores_tr['confusion'])
  if (scores_te['f1'] > 0 and scores_te['accuracy']) > 0 :
    print(f"> ** Confusion matrix (Test)**")
    print(scores_te['confusion'])

    
def calculate_train_and_test_scores(predictor, X_train, y_train, X_test=MyEmptydf, y_test=MyEmptydf):
  predictions = predictor.predict(X_train)
  train_scores = calculate_scores(predictions, y_train)

  # Only show y_test score if we passed in y_test
  if y_test.empty:
    test_scores = calculate_scores(~y_train, y_train)
    f1_delta = 0.0 # not considering
  else:  
    predictions = predictor.predict(X_test)
    test_scores = calculate_scores(predictions, y_test)
    f1_delta = train_scores['f1'] - test_scores['f1']
  
  return {
    'f1_delta': f1_delta,
    'train_scores': train_scores,
    'test_scores': test_scores,
  }


def extract_grid_params(cv_results_params_dict):
  old_params = collections.OrderedDict(sorted(cv_results_params_dict.items()))
  
  extrated_params = {}
  for param, value in old_params.items():
    if param == 'clf__estimator':
      extrated_params['>estimator<'] = class_to_string(value[0])
      continue
      
    param = param.replace('clf__estimator__', '')
    extrated_params[param] = value[0]
  
  return extrated_params


def train_models(X, y, pipeline, param_grid, cv, scoring):
  grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring=scoring, return_train_score=True, iid=False, n_jobs=-1)
  grid_search.fit(X, y)
  return grid_search
  

def score_best_models(cv_results, X_train, y_train, X_test, y_test, pipeline, param_grid, cv, scoring,
                     # mean_test_score_threshold=0.84, f1_delta_threshold=0.025):
                       mean_test_score_threshold=0.5, f1_delta_threshold=1): # we cannot use f1_delta to filter models as it is using the test data
  scores = {}
  index = -1
  for rank in cv_results['rank_test_score']:
    index += 1
    if cv_results['mean_test_score'][index] >= mean_test_score_threshold:
      param_grid = cv_results['params'][index].copy()
      for key, value in param_grid.items(): param_grid[key] = [value] 
      grid_search = train_models(X_train, y_train, pipeline, param_grid, cv, scoring)
      
      grid_pipeline = grid_search.best_estimator_
      estimator_name = class_to_string(grid_pipeline['clf'].estimator)
      estimator_scores = calculate_train_and_test_scores(grid_pipeline, X_train, y_train, X_test=X_test, y_test=y_test)
      
      if (abs(estimator_scores['f1_delta'])) <= f1_delta_threshold:
        scores[f'{index} - {estimator_name}'] = {
          'estimator': grid_pipeline,
          'params'   : extract_grid_params(param_grid),
          'scores'   : estimator_scores,
        }
  return scores


def print_always_true_scores(y_train, y_test):
  print('\n\n>>>>>> Comparing with our base estimator (always predicts true):')
  scores_tr = calculate_scores(np.ones(y_train.shape), y_train )
  scores_te = calculate_scores(np.ones(y_test.shape), y_test )
  print_scores(scores_tr, scores_te)

  
def train_models_and_select_best(X_train, y_train, X_test, y_test, pipeline, param_grid, cv=CV_COUNT, scoring=SCORING_METRICS, print_params=True):
  grid_search = train_models(X_train, y_train, pipeline, param_grid, cv, scoring)
  
  print('\n>>>>>> How each estimator is scoring:')
  print(f"Rank/position:   {grid_search.cv_results_['rank_test_score']}")
  print(f"Mean test score: {grid_search.cv_results_['mean_test_score']}")
  
  print('\n\n>>>>>> Auto-selecting the best performers')
  best_models = score_best_models(grid_search.cv_results_, X_train, y_train, X_test, y_test, pipeline, param_grid, cv, scoring)
  
  print(' Sensitivity/recall: how good the model is at detecting the positives')
  print(' Specificity       : how good the model is at avoiding false alarms')
  print(' Precision         : how many of the positively classified were relevant')
  
  for model_name, params in best_models.items():
    print(f"\nModel: {model_name}")
    if print_params: pp.pprint(params['params'])
    score = params['scores']
    print(f"> F1 delta (train-test): {score['f1_delta']:f}")
    print_scores(score['train_scores'], score['test_scores'])
  
  print_always_true_scores(y_train, y_test)

  return best_models, grid_search


def pick_top_n_models(cv_results, max_models, X_train, y_train, pipeline, param_grid, cv, scoring):
  scores = {}
  index = -1
  
  print("We also reject the models who never predicts any positive or negative case correctly.")
  for rank in cv_results['rank_test_score']:
    index += 1
    # only show models what is on the top max_models
    if rank <= max_models:

      param_grid = cv_results['params'][index].copy()
      for key, value in param_grid.items(): param_grid[key] = [value]

      grid_search_temp = train_models(X_train, y_train, pipeline, param_grid, cv, scoring)

      grid_pipeline = grid_search_temp.best_estimator_
      estimator_name = class_to_string(grid_pipeline['clf'].estimator) + ' (Rank ' + str(rank) + ')'
      estimator_scores = calculate_train_and_test_scores(grid_pipeline, X_train, y_train)
      # only want those with specificity > 0, otherwise we will pick models that never predict correctly any false
      if ((estimator_scores['train_scores']['specificity'] > 0) and (estimator_scores['train_scores']['precision'] > 0)):   
        scores[f'{index} - {estimator_name}'] = {
          'estimator': grid_pipeline,
          'params'   : extract_grid_params(param_grid),
          'scores'   : estimator_scores,
        }
      else:
        print ("Reject model {0} {1} (precision:{2}, specificity:{3})".format(index, 
                                                                 estimator_name,                         
                                                                 estimator_scores['train_scores']['precision'],
                                                                 estimator_scores['train_scores']['specificity']))
  return scores


def train_and_select_any_top_n(X_train, y_train, pipeline, param_grid, max_models=PICK_TOP_N_MODELS, cv=CV_COUNT, scoring=SCORING_METRICS, print_params=True):
  grid_search = train_models(X_train, y_train, pipeline, param_grid, cv, scoring)
 
  print('\n>>>>>> How each estimator is scoring:')
  print(f"Rank/position:   {grid_search.cv_results_['rank_test_score']}")
  print(f"Mean test score: {grid_search.cv_results_['mean_test_score']}")
    
  print(' Sensitivity/recall: how good the model is at detecting the positives')
  print(' Specificity       : how good the model is at avoiding false alarms')
  print(' Precision         : how many of the positively classified were relevant')
  
  print('\n\n>>>>>>showing top {} models'.format(max_models) )
  best_models = pick_top_n_models(grid_search.cv_results_, max_models, X_train, y_train, pipeline, param_grid, cv, scoring)
  for model_name, params in best_models.items():
    print(f"\nModel: {model_name}")
    if print_params: pp.pprint(params['params'])
    score = params['scores']
    print(f"> F1 delta (train-test): {score['f1_delta']:f}")
    print_scores(score['train_scores'], score['test_scores'])
  
  print('\n\n>>>>>> Comparing with our base estimator (always predicts true):')
  scores_tr = calculate_scores(np.ones(y_train.shape), y_train )
  scores_te = calculate_scores(np.ones(y_test.shape), y_test )
  print_scores(scores_tr, scores_te)

  return best_models, grid_search


from sklearn.ensemble import VotingClassifier
def creating_VotingClassifier(clf_list, option):
  temp_list = []
  for best_model_name, best_model_dict in clf_list.items():
    temp_list.append((best_model_name, best_model_dict['estimator']['clf'].estimator))
  return (VotingClassifier(estimators=temp_list, voting=option))

In [7]:
param_grid_all_features = [
  # SVC has the tuned parameters
  {
    'clf__estimator': [SVC()],
    'clf__estimator__random_state': [RANDOM_SEED],
    'clf__estimator__probability': [True],
    'clf__estimator__kernel': ["poly"],
    'clf__estimator__degree' : [3], 
    'clf__estimator__gamma': ['scale'],  
    'clf__estimator__C': [0.1],
    'clf__estimator__decision_function_shape': ['ovo'],      
  },
  # RandomForestClassifier has the tuned parameters
  {
    'clf__estimator': [RandomForestClassifier()],
    'clf__estimator__random_state':      [RANDOM_SEED],
    'clf__estimator__bootstrap':         [True],
    'clf__estimator__class_weight':      [None],
    'clf__estimator__criterion':         ['entropy'],
    'clf__estimator__max_depth':         [2],
    'clf__estimator__max_features':      ['auto'], 
    'clf__estimator__n_estimators':      [20],
  },
  # LogisticRegression has tuned parameters  
  {
    'clf__estimator': [LogisticRegression()],
    'clf__estimator__random_state': [RANDOM_SEED],
    'clf__estimator__C': [0.9],
    'clf__estimator__multi_class': ['ovr'],
    'clf__estimator__solver': ['lbfgs'],
    'clf__estimator__max_iter': [1000],      
  },      
  #  DecisionTreeClassifier has tuned parameters 
  {
    'clf__estimator': [DecisionTreeClassifier()],
    'clf__estimator__random_state': [RANDOM_SEED],
    'clf__estimator__criterion':    ['entropy'],
    'clf__estimator__max_depth':    [2],
  },   
 
#   {
#     'clf__estimator': [GradientBoostingClassifier()],
#     'clf__estimator__random_state': [RANDOM_SEED],
#   },
  #  GradientBoostingClassifier has tuned parameters   
  {
    'clf__estimator': [GradientBoostingClassifier()],
    'clf__estimator__random_state': [RANDOM_SEED],
    'clf__estimator__loss': ['exponential'],
    'clf__estimator__learning_rate':     [5],
    'clf__estimator__n_estimators' : [100],
    'clf__estimator__max_depth' : [5],
    'clf__estimator__max_features' :['auto']
  },   
#   {    
#     'clf__estimator': [XGBClassifier()],
#     'clf__estimator__random_state':      [RANDOM_SEED],
#   },
    
  # XGBClassifier with tuned parameters
  {
    'clf__estimator': [XGBClassifier()],
    'clf__estimator__random_state':      [RANDOM_SEED],
    'clf__estimator__max_depth':         [3],
    'clf__estimator__learning_rate':     [10],
    'clf__estimator__n_estimators':      [20], 
    'clf__estimator__booster':           ['gbtree'], 
  },

  # KNeighborsClassifier has tuned parameters  
  {
    'clf__estimator': [KNeighborsClassifier()],
    'clf__estimator__n_neighbors': [4],
  },
  #   AdaBoostClassifier has the tuned parameters
  {
    'clf__estimator': [AdaBoostClassifier()],
    'clf__estimator__random_state': [RANDOM_SEED],
    'clf__estimator__n_estimators':      [100],
    'clf__estimator__learning_rate':     [0.1],
    'clf__estimator__algorithm': ['SAMME.R'],
  },       
  {
    'clf__estimator': [GaussianNB()],
  },   
]

# We use the mean_score of cross_validation to select models because we have very few samples in the dataset and if we split further
# for test, model_sel, test, we won't be able to train the model properly.
# best_models1, grid_search1 = train_models_and_select_best(X_train, y_train, X_test, y_test, training_pipeline, param_grid_all_features, cv=CV_COUNT, scoring=SCORING_METRICS)

# find the best n models 
best_models1, grid_search1 = train_and_select_any_top_n(X_train, y_train,training_pipeline, param_grid_all_features, cv=CV_COUNT, scoring=SCORING_METRICS)


>>>>>> How each estimator is scoring:
Rank/position:   [9 3 8 4 6 2 7 5 1]
Mean test score: [0.17777778 0.46805007 0.22539683 0.39692982 0.32206119 0.61206349
 0.27961973 0.33108171 0.65151515]
 Sensitivity/recall: how good the model is at detecting the positives
 Specificity       : how good the model is at avoiding false alarms
 Precision         : how many of the positively classified were relevant


>>>>>>showing top 5 models
We also reject the models who never predicts any positive or negative case correctly.
Reject model 5 XGBClassifier (Rank 2) (precision:0.5074626865671642, specificity:0.0)

Model: 1 - RandomForestClassifier (Rank 3)
{'>estimator<': 'RandomForestClassifier',
 'bootstrap': True,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 'auto',
 'n_estimators': 20,
 'random_state': 1}
> F1 delta (train-test): 0.000000
> Scores     <train>    | <NoTest>
> F1         : 0.888889 | 0.000000
> Precision  : 0.965517 | 0.000000
> Recall     : 0.

### Now create a VotingClassifier with the top n models

In [8]:
# Create a pipeline for using the voting classifiers
allfeatures_pipeline = Pipeline(verbose=False, steps=[
       ('pol', PolynomialFeatures(degree = POLYNOMIAL_DEGREE)),
       ('nor', Normalizer()),
       ('clf', creating_VotingClassifier(best_models1, "soft") )
      ])

#print_always_true_scores(y_train, y_test)
#print("============== ")
allfeatures_pipeline.fit(X_train, y_train)
print("All features score:", allfeatures_pipeline.score(X_test,y_test)) 
y_tr_pred = allfeatures_pipeline.predict(X_train)
y_te_pred = allfeatures_pipeline.predict(X_test)
scores_tr = calculate_scores(y_tr_pred, y_train)
scores_te = calculate_scores(y_te_pred, y_test)
print_scores(scores_tr, scores_te)


All features score: 0.8235294117647058
> Scores     <train>    | <test>
> F1         : 0.850000 | 0.857143
> Precision  : 0.739130 | 0.750000
> Recall     : 1.000000 | 1.000000
> Specificity: 0.636364 | 0.625000
> Accuracy   : 0.820896 | 0.823529
> AUC        : 0.818182 | 0.812500
> ** Confusion matrix (Train)**
[[21 12]
 [ 0 34]]
> ** Confusion matrix (Test)**
[[5 3]
 [0 9]]


Our best results (that is not overfit for training data):

Top models: 
- GaussianNB (Rank 1)
- (Rejected) XGBClassifier (Rank 2) 
- RandomForestClassifier (Rank 3)
- DecisionTreeClassifier (Rank 4)
- AdaBoostClassifier (Rank 5)



```
* SCORING_METRICS = 'f1'
* PICK_TOP_N_MODELS = 5
* CV_COUNT = 3 
* POLYNOMIAL_DEGREE = 2

All features score: 0.8235294117647058
> Scores     <train>    | <test>
> F1         : 0.850000 | 0.857143
> Precision  : 0.739130 | 0.750000
> Recall     : 1.000000 | 1.000000
> Specificity: 0.636364 | 0.625000
> Accuracy   : 0.820896 | 0.823529
> AUC        : 0.818182 | 0.812500
  
 ** Confusion matrix (Train)**
[[20 13]
 [ 0 34]]
 ** Confusion matrix (Test)**
[[4 4]
 [0 9]]

```


  
